In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, concatenate

In [2]:
# Load CSV file
csv_file = "glaucoma.csv"
data = pd.read_csv(csv_file)
data.head()

,Filename,ExpCDR,Eye,Set,Glaucoma
0,001.jpg,0.7097,OD,A,0
1,002.jpg,0.6953,OS,A,0
2,003.jpg,0.9629,OS,A,0
3,004.jpg,0.7246,OD,A,0
4,005.jpg,0.6138,OS,A,0


In [3]:
data.describe(include='all')

,Filename,ExpCDR,Eye,Set,Glaucoma
count,650,650.000000,650,650,650.000000
unique,650,NaN,2,2,NaN
top,001.jpg,NaN,OD,A,NaN
freq,1,NaN,327,325,NaN
mean,NaN,0.576308,NaN,NaN,0.258462
std,NaN,0.116395,NaN,NaN,0.438126
min,NaN,0.161000,NaN,NaN,0.000000
25%,NaN,0.496675,NaN,NaN,0.000000
50%,NaN,0.567650,NaN,NaN,0.000000
75%,NaN,0.648475,NaN,NaN,1.000000


In [4]:
print(data['Eye'].unique())
print(data['Set'].unique())
print(data['Glaucoma'].unique())

['OD' 'OS']
['A' 'B']
[0 1]


In [5]:
data['Eye'] = data['Eye'].map({'OD': 0, 'OS': 1})
data.head()

,Filename,ExpCDR,Eye,Set,Glaucoma
0,001.jpg,0.7097,0,A,0
1,002.jpg,0.6953,1,A,0
2,003.jpg,0.9629,1,A,0
3,004.jpg,0.7246,0,A,0
4,005.jpg,0.6138,1,A,0


In [6]:
data['Set'] = data['Set'].map({'A': 0, 'B': 1})
data.head()

,Filename,ExpCDR,Eye,Set,Glaucoma
0,001.jpg,0.7097,0,0,0
1,002.jpg,0.6953,1,0,0
2,003.jpg,0.9629,1,0,0
3,004.jpg,0.7246,0,0,0
4,005.jpg,0.6138,1,0,0


## Important

Please note that the file structure for the dataset should be as follows for the below logic to work correctly.

```
./
|
|- Dataset-new
|		|- Merged
|		       | - <all-the-images>  # important!
|- Dataset
|	  	|- Train
|		|	|- Glaucoma-Positive
|		|	|		|- <all-the-positive-images>
|		|	|- Glaucoma-Negative
|		|			|- <all-the-negative-images>
|	  	|- Validation
|		 	|- Glaucoma-Positive
|		 	|		|- <all-the-positive-images>
|		 	|- Glaucoma-Negative
|		 			|- <all-the-negative-images>
|- glaucoma.csv
|- <this-notebook>
```

In [7]:
# Image directory
image_dir = "Dataset-new/Merged"

In [8]:
# Function to load images and labels
def load_data(data, image_dir, image_column="Filename"):
    images = []
    features = []

    for index, row in data.iterrows():
        image_path = os.path.join(image_dir, row[image_column])
        image = load_img(image_path, target_size=(150, 150))  # Resize image to (150, 150)
        image = img_to_array(image)
        images.append(image)

        feature_exp_cdr = row["ExpCDR"]
        feature_type = row["Eye"]
        feature_set = row["Set"]
        features.append([feature_exp_cdr, feature_type, feature_set])

    images = np.array(images)
    features = np.array(features)

    return images, features

In [9]:
# Load images and features
images, features = load_data(data, image_dir)

In [10]:
# Normalize pixel values
images = images / 255.0

In [11]:
X_images_train, X_images_test, X_features_train, X_features_test, y_train, y_test = train_test_split(
    images, features, data["Glaucoma"], test_size=0.2, random_state=42)

In [12]:
# Define CNN model
input_image = Input(shape=(150, 150, 3))
conv1 = Conv2D(32, (3, 3), activation='relu')(input_image)
pool1 = MaxPooling2D(2, 2)(conv1)
conv2 = Conv2D(64, (3, 3), activation='relu')(pool1)
pool2 = MaxPooling2D(2, 2)(conv2)
conv3 = Conv2D(128, (3, 3), activation='relu')(pool2)
pool3 = MaxPooling2D(2, 2)(conv3)
conv4 = Conv2D(128, (3, 3), activation='relu')(pool3)
pool4 = MaxPooling2D(2, 2)(conv4)
flatten = Flatten()(pool4)

In [13]:
input_features = Input(shape=(3,))
concatenated = concatenate([flatten, input_features])

In [14]:
dense1 = Dense(512, activation='relu')(concatenated)
output = Dense(1, activation='sigmoid')(dense1)

In [15]:
model = Model(inputs=[input_image, input_features], outputs=output)

In [16]:
model.compile(optimizer='adam', loss='binary_crossentropy',
              metrics=['accuracy'])

In [17]:
model.fit([X_images_train, X_features_train], y_train, epochs=100,
          batch_size=32, validation_data=([X_images_test, X_features_test], y_test))

Epoch 1/100


2024-04-11 11:53:10.290926: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


17/17 [==============================] - 5s 250ms/step - loss: 0.6152 - accuracy: 0.7423 - val_loss: 0.5778 - val_accuracy: 0.7385
Epoch 2/100
17/17 [==============================] - 5s 274ms/step - loss: 0.5733 - accuracy: 0.7423 - val_loss: 0.5736 - val_accuracy: 0.7385
Epoch 3/100
17/17 [==============================] - 4s 220ms/step - loss: 0.5722 - accuracy: 0.7423 - val_loss: 0.5991 - val_accuracy: 0.7385
Epoch 4/100
17/17 [==============================] - 4s 228ms/step - loss: 0.6151 - accuracy: 0.7423 - val_loss: 0.5835 - val_accuracy: 0.7385
Epoch 5/100
17/17 [==============================] - 4s 219ms/step - loss: 0.5733 - accuracy: 0.7423 - val_loss: 0.5840 - val_accuracy: 0.7385
Epoch 6/100
17/17 [==============================] - 4s 222ms/step - loss: 0.5737 - accuracy: 0.7423 - val_loss: 0.6009 - val_accuracy: 0.7385
Epoch 7/100
17/17 [==============================] - 4s 212ms/step - loss: 0.5780 - accuracy: 0.7423 - val_loss: 0.5658 - val_accuracy: 0.7385
Epoch 8/100

In [19]:
# Evaluate the model
loss, accuracy = model.evaluate([X_images_test, X_features_test], y_test)
print("Test Accuracy:", accuracy)

5/5 [==============================] - 0s 80ms/step - loss: 1.6633 - accuracy: 0.6385
Test Accuracy: 0.6384615302085876


# testing

In [25]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array


# Function to preprocess image
def preprocess_image(image_path):
    image = load_img(image_path, target_size=(
        150, 150))  # Resize image to (150, 150)
    image = img_to_array(image)
    image = image / 255.0  # Normalize pixel values
    return image

# Function to preprocess additional attributes


def preprocess_attributes(exp_cdr, type, set):
    # Perform any necessary preprocessing here
    # For example, you might normalize or encode categorical variables
    # You should ensure that the preprocessing steps are consistent with what was done during training
    processed_attributes = [exp_cdr, type, set]
    return processed_attributes


# 001.jpg,0.7097,OD,A,0
# 036.jpg,0.5097,OD,A,1
sample_data_set = [
    ('Dataset-new/Merged/001.jpg', 0.7097, 0, 0, 0),
    ('Dataset-new/Merged/036.jpg', 0.5097, 0, 0, 1),
]

for data in sample_data_set:
    # Example usage: predict on a single image with additional attributes

    image_path, exp_cdr, type, set, actual = data

    preprocessed_image = preprocess_image(image_path)
    preprocessed_attributes = preprocess_attributes(exp_cdr, type, set)

    # Make prediction
    prediction = model.predict(
        [np.array([preprocessed_image]), np.array([preprocessed_attributes])])
    # This will give you the predicted probability of the image being in the positive class
    print(f"\nActual: {actual}, Prediction: {prediction[0][0]}\n")

1/1 [==============================] - 0s 89ms/step

Actual: 0, Prediction: 0.00016108265845105052

1/1 [==============================] - 0s 17ms/step

Actual: 1, Prediction: 0.996231734752655



In [26]:
model.save("glaucoma_detection_model_vgg.h5")